In [4]:
import os
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta
import firebirdsql

# Função para obter a conexão com o banco de dados Firebird
def get_firebird_connection():
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

# Define o período: de 01/03/2024 a 28/02/2025
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2025, 2, 28)

# Conecta ao banco e extrai os dados da tabela APAGAR
conn = get_firebird_connection()
query = """
    SELECT DTVENCIMENTO, VALOR
    FROM APAGAR
    WHERE DTVENCIMENTO BETWEEN ? AND ?
"""
df = pd.read_sql(query, conn, params=(start_date, end_date))
conn.close()

# Converte a coluna DTVENCIMENTO para datetime (assumindo formato dd/mm/yy)
df['DTVENCIMENTO'] = pd.to_datetime(df['DTVENCIMENTO'], dayfirst=True)

# Cria uma coluna 'day' com o dia do mês (1 a 31)
df['day'] = df['DTVENCIMENTO'].dt.day

# Agrupa os dados por 'day' e calcula a média dos valores para cada dia
avg_by_day = df.groupby('day')['VALOR'].mean().reset_index()
avg_by_day = avg_by_day.rename(columns={'VALOR': 'avg_valor'})

# Encontra o dia com a menor média de pagamento
min_day = avg_by_day.loc[avg_by_day['avg_valor'].idxmin()]

print("Média de pagamento por dia do mês (geral):")
print(avg_by_day)

print("\nDia com menor pagamento em média:")
print(f"Dia {min_day['day']} com média de {min_day['avg_valor']:.2f}")

/var/folders/gt/g493gb1j08q58kpxyclbc6c40000gn/T/ipykernel_79340/3902934038.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(start_date, end_date))


Média de pagamento por dia do mês (geral):
    day    avg_valor
0     1   720.579845
1     2   917.331538
2     3  1091.563071
3     4  1676.692460
4     5  1391.715070
5     6   965.017093
6     7  1066.531159
7     8   785.277941
8     9   750.993881
9    10   687.664363
10   11   598.510147
11   12   500.542218
12   13   457.009687
13   14   556.385095
14   15   871.700273
15   16   703.902933
16   17   839.551978
17   18   876.215526
18   19   965.893684
19   20  1202.480663
20   21   457.040253
21   22   708.651453
22   23   511.242870
23   24   712.180667
24   25   984.285969
25   26   754.655275
26   27   734.816762
27   28   823.596855
28   29   717.620685
29   30   605.639755
30   31   733.544632

Dia com menor pagamento em média:
Dia 13.0 com média de 457.01
